In [1]:
import os

os.environ["XDG_SESSION_TYPE"] = "x11"
os.environ.pop("WAYLAND_DISPLAY", None)

import torch
import numpy as np
import pandas as pd

from pathlib import Path
from dotenv import load_dotenv

from EHydro_TreeUnet.trainers import TreeProjectorTrainer
from torchsparse.nn import functional as F

F.set_kmap_mode("hashmap_on_the_fly")

In [ ]:
load_dotenv()

TREE_PROJECTOR_DIR = Path(os.environ.get('TREE_PROJECTOR_DIR', Path.home() / 'tree_projector'))
DATASET_FOLDER = 'MixedDataset'
VERSION_NAME = 'tree_projector_VS-0.2_DA-48_E-3_V2'

VOXEL_SIZE = 0.2
# FEAT_KEYS = ['intensity', 'x_norm', 'y_norm', 'z_norm']
FEAT_KEYS = ['intensity']
CENTROID_SIGMA = 1.5
TRAIN_PCT = 0.9
DATA_AUGMENTATION_COEF = 48
YAW_RANGE = (0.0, 360.0)
TILT_RANGE = (-5.0, 5.0)
SCALE_RANGE = (0.9, 1.2)

TRAINING = True
TEST_LR = []
EPOCHS = 3
START_ON_EPOCH = 0
BATCH_SIZE = 1
SEMANTIC_LOSS_COEF = 2.0
CENTROID_LOSS_COEF = 1.0
OFFSET_LOSS_COEF = 4.0
INSTANCE_LOSS_COEF = 1.0
BACKBONE_LR = 1e-3
SEMANTIC_LR = 4e-3
OFFSET_LR = 2e-3
CENTROID_LR = 2e-3
INSTANCE_LR = 1e-2

RESNET_BLOCKS = [
    (3, 32, 3, 1),
    (3, 64, 3, 2),
    (3, 128, 3, 2),
    (3, 256, 3, 2),
    (1, 256, (1, 1, 3), (1, 1, 2)),
]
INSTANCE_DENSITY = 0.01
SCORE_THRES = 0.1
CENTROID_THRES = 0.2
DESCRIPTOR_DIM = 32

CHARTS_IGNORE_CLASS = []

In [3]:
def smooth(arr: np.ndarray, window: int) -> np.ndarray:
    if window <= 1:
        return arr

    kernel = np.ones(window, dtype=float)
    if arr.ndim == 1:
        denom = np.convolve(np.ones_like(arr), kernel, mode="same")
        return np.convolve(arr, kernel, mode="same") / denom

    # 2-D: suavizar cada columna por separado
    smoothed = np.empty_like(arr, dtype=float)
    denom = np.convolve(np.ones(arr.shape[0]), kernel, mode="same")
    for c in range(arr.shape[1]):
        smoothed[:, c] = np.convolve(arr[:, c], kernel, mode="same") / denom
    return smoothed
        
def gen_charts(trainer, losses, stats, training: bool, window: int = 1, ignore_class = []):
    keys = stats[0].keys()
    stats = {k: np.array([d[k] for d in stats]) for k in keys}

    loss = np.asarray(losses)
    loss_s = np.clip(smooth(loss[:, 0], window), 0.0, 10.0)
    loss_sem_s = np.clip(smooth(loss[:, 1], window), 0.0, 10.0)
    loss_centroid_s = np.clip(smooth(loss[:, 2], window), 0.0, 10.0)
    loss_inst_s = np.clip(smooth(loss[:, 3], window), 0.0, 10.0)

    iou_semantic = smooth(stats['iou_semantic'], window)
    mean_iou_semantic = smooth(stats['mean_iou_semantic'], window)
    precision_semantic = smooth(stats['precision_semantic'], window)
    mean_precision_semantic = smooth(stats['mean_precision_semantic'], window)
    recall_semantic = smooth(stats['recall_semantic'], window)
    mean_recall_semantic = smooth(stats['mean_recall_semantic'], window)
    f1_semantic = smooth(stats['f1_semantic'], window)
    mean_f1_semantic = smooth(stats['mean_f1_semantic'], window)

    mean_iou_instance = smooth(stats['mean_iou_instance'], window)

    # --- Global loss -----------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(loss_s, label=f"{'Training' if training else 'Inference'} Loss (MA{window})")
    plt.xlabel("Step"); plt.ylabel("Loss")
    plt.title(f"Loss evolution during {'Training' if training else 'Inference'}")
    plt.legend(); plt.grid(True); plt.show()

    # --- Semantic loss ----------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(loss_sem_s, label=f"{'Training' if training else 'Inference'} Loss (MA{window})")
    plt.xlabel("Step"); plt.ylabel("Loss")
    plt.title(f"Semantic loss evolution during {'Training' if training else 'Inference'}")
    plt.legend(); plt.grid(True); plt.show()

    # --- Centroid loss ----------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(loss_centroid_s, label=f"{'Training' if training else 'Inference'} Loss (MA{window})")
    plt.xlabel("Step"); plt.ylabel("Loss")
    plt.title(f"Centroid loss evolution during {'Training' if training else 'Inference'}")
    plt.legend(); plt.grid(True); plt.show()

    # --- Instance loss ----------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(loss_inst_s, label=f"{'Training' if training else 'Inference'} Loss (MA{window})")
    plt.xlabel("Step"); plt.ylabel("Loss")
    plt.title(f"Instance loss evolution during {'Training' if training else 'Inference'}")
    plt.legend(); plt.grid(True); plt.show()

    # --- Semantic IoU -----------------------------------------------------
    plt.figure(figsize=(10, 5))
    for c in range(trainer.dataset.num_classes):
        if trainer.dataset.class_names[c] in ignore_class:
            continue
        
        plt.plot(iou_semantic[:, c], label=trainer.dataset.class_names[c])
    plt.xlabel("Step"); plt.ylabel("IoU")
    plt.title(f"Semantic IoU evolution during {'Training' if training else 'Inference'} (MA{window})")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- Semantic mIoU ----------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(mean_iou_semantic, label=f"{'Training' if training else 'Inference'} mIoU (MA{window})")
    plt.xlabel("Step"); plt.ylabel("mIoU")
    plt.title(f"Semantic mIoU evolution during {'Training' if training else 'Inference'}")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    # --- Instance mIoU ----------------------------------------------------
    plt.figure(figsize=(10, 5))
    plt.plot(mean_iou_instance, label=f"{'Training' if training else 'Inference'} Instance mIoU (MA{window})")
    plt.xlabel("Step"); plt.ylabel("mIoU")
    plt.title(f"Instance mIoU evolution during {'Training' if training else 'Inference'}")
    plt.ylim(0, 1)
    plt.legend(); plt.grid(True); plt.show()

    column_names = ['IoU', 'Precision', 'Recall', 'F1']
    row_names = [trainer.dataset.class_names[c] for c in range(trainer.dataset.num_classes) if trainer.dataset.class_names[c] not in ignore_class]
    row_names.append('Mean')

    iou_semantic_arr = stats['iou_semantic']
    prec_arr = stats['precision_semantic']
    recall_arr = stats['recall_semantic']
    f1_arr = stats['f1_semantic']

    data = [
        [iou_semantic_arr[:, c].mean(), prec_arr[:, c].mean(), recall_arr[:, c].mean(), f1_arr[:, c].mean()]
    for c in range(trainer.dataset.num_classes) if trainer.dataset.class_names[c] not in ignore_class]

    means = np.array(data).mean(axis=0)
    data.append(list(means))

    df = pd.DataFrame(data, columns=column_names, index=row_names)
    display(df)

In [4]:
trainer = TreeProjectorTrainer(
    tree_projector_dir=TREE_PROJECTOR_DIR,
    dataset_folder=DATASET_FOLDER,
    version_name=VERSION_NAME,

    voxel_size=VOXEL_SIZE,
    feat_keys=FEAT_KEYS,
    centroid_sigma=CENTROID_SIGMA,
    train_pct=TRAIN_PCT,
    data_augmentation_coef=DATA_AUGMENTATION_COEF,
    yaw_range=YAW_RANGE,
    tilt_range=TILT_RANGE,
    scale_range=SCALE_RANGE,

    training=TRAINING,
    test_lr=TEST_LR,
    epochs=EPOCHS,
    start_on_epoch=START_ON_EPOCH,
    batch_size=BATCH_SIZE,
    semantic_loss_coef=SEMANTIC_LOSS_COEF,
    centroid_loss_coef=CENTROID_LOSS_COEF,
    offset_loss_coef=OFFSET_LOSS_COEF,
    instance_loss_coef=INSTANCE_LOSS_COEF,
    backbone_lr=BACKBONE_LR,
    semantic_lr=SEMANTIC_LR,
    offset_lr=OFFSET_LR,
    centroid_lr=CENTROID_LR,
    instance_lr=INSTANCE_LR,

    resnet_blocks=RESNET_BLOCKS,
    instance_density=INSTANCE_DENSITY,
    score_thres=SCORE_THRES,
    centroid_thres=CENTROID_THRES,
    descriptor_dim=DESCRIPTOR_DIM
)

if TRAINING:
    trainer.train()


import open3d as o3d
import matplotlib.pyplot as plt

pcd_gt = o3d.geometry.PointCloud()
pcd_pred = o3d.geometry.PointCloud()

for eval_result in trainer.eval():
    batch_idx = eval_result['semantic_output'].C.cpu().numpy()[:, 0]
    centroid_batch_idx = eval_result['centroid_confidence_output'].C.cpu().numpy()[:, 0]

    voxels = eval_result['semantic_output'].C.cpu().numpy()[:, 1:]
    centroid_voxels = eval_result['centroid_confidence_output'].C.cpu().numpy()[:, 1:]

    semantic_output = torch.argmax(eval_result['semantic_output'].F.cpu(), dim=1).numpy()
    semantic_labels = eval_result['semantic_labels'].F.cpu().numpy()

    semantic_mask = semantic_labels != 0
    centroid_score_output = np.zeros((voxels.shape[0], 1), dtype=float)
    centroid_score_output[semantic_mask] = eval_result['centroid_score_output'].F.cpu().numpy()
    centroid_score_labels = np.zeros((voxels.shape[0], 1), dtype=float)
    centroid_score_labels[semantic_mask] = eval_result['centroid_score_labels'].F.cpu().numpy()

    centroid_confidence_output = eval_result['centroid_confidence_output'].F.cpu().numpy()

    offset_output = np.zeros((voxels.shape[0], 3), dtype=float)
    offset_output[semantic_mask] = eval_result['offset_output'].F.cpu().numpy()
    offset_output = np.sign(offset_output) * np.expm1(np.abs(offset_output))
    offset_labels = np.zeros((voxels.shape[0], 3), dtype=float)
    offset_labels[semantic_mask] = eval_result['offset_labels'].F.cpu().numpy()
    offset_labels = np.sign(offset_labels) * np.expm1(np.abs(offset_labels))

    instance_output_remap_logits = torch.zeros((eval_result['instance_output'].F.size(0), eval_result['remap_info']['num_instances']), device=eval_result['instance_output'].F.device, dtype=eval_result['instance_output'].F.dtype)
    instance_output_remap_logits[:, eval_result['remap_info']['gt_indices']] = eval_result['instance_output'].F[:, eval_result['remap_info']['pred_indices']]

    instance_output = np.zeros((voxels.shape[0],), dtype=int)
    instance_output[semantic_mask] = torch.argmax(eval_result['instance_output'].F.cpu(), dim=1).numpy()
    instance_labels = np.zeros((voxels.shape[0],), dtype=int)
    instance_labels[semantic_mask] = eval_result['instance_labels'].F.cpu().numpy()

    instance_output_remap = np.zeros((voxels.shape[0],), dtype=int)
    instance_output_remap[semantic_mask] = torch.argmax(instance_output_remap_logits.cpu(), dim=1).numpy()

    rng = np.random.default_rng(0)
    palette = rng.random((len(np.unique(instance_labels)), 3))

    id2color = {uid: palette[i] for i, uid in enumerate(np.unique(instance_labels))}

    for idx in np.unique(batch_idx):
        mask = batch_idx == idx
        cloud_voxels = voxels[mask]

        cloud_semantic_output = semantic_output[mask]
        cloud_semantic_labels = semantic_labels[mask]

        cloud_centroid_score_output = centroid_score_output[mask]
        cloud_centroid_score_labels = centroid_score_labels[mask]

        cloud_offset_output = offset_output[mask]
        cloud_offset_labels = offset_labels[mask]

        cloud_instance_output = instance_output[mask]
        cloud_instance_output_remap = instance_output_remap[mask]
        cloud_instance_labels = instance_labels[mask]

        mask = centroid_batch_idx == idx
        cloud_centroid_voxels = centroid_voxels[mask]
        cloud_centroid_confidence_output = centroid_confidence_output[mask]

        pcd_gt.points = o3d.utility.Vector3dVector(cloud_voxels)
        pcd_gt.translate((20 / VOXEL_SIZE, 0, 0))
        pcd_pred.points = o3d.utility.Vector3dVector(cloud_voxels)

        colors = trainer.dataset.class_colormap[cloud_semantic_labels] / 255.0
        pcd_gt.colors = o3d.utility.Vector3dVector(colors)
        colors = trainer.dataset.class_colormap[cloud_semantic_output] / 255.0
        pcd_pred.colors = o3d.utility.Vector3dVector(colors)
        o3d.visualization.draw_geometries([pcd_pred, pcd_gt])

        cmap = plt.get_cmap('viridis')
        cmap_spheres = plt.get_cmap('inferno')
        colors = cmap(cloud_centroid_score_labels[:, 0])[:, :3]
        pcd_gt.colors = o3d.utility.Vector3dVector(colors)
        colors = cmap(cloud_centroid_score_output[:, 0])[:, :3]
        pcd_pred.colors = o3d.utility.Vector3dVector(colors)

        spheres = []
        for i in np.unique(cloud_instance_output):
            center = centroid_voxels[i]
            confidence = centroid_confidence_output[i][0]

            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=1.5)
            sphere.translate(center)
            color = cmap_spheres(confidence)[:3]
            sphere.paint_uniform_color(color)
            spheres.append(sphere)

        o3d.visualization.draw_geometries([pcd_pred, pcd_gt] + spheres)

        voxels_disp_output = cloud_voxels + cloud_offset_output
        voxels_disp_labels = cloud_voxels + cloud_offset_labels

        pcd_gt.points = o3d.utility.Vector3dVector(voxels_disp_labels)
        pcd_gt.translate((20 / VOXEL_SIZE, 0, 0))
        pcd_pred.points = o3d.utility.Vector3dVector(voxels_disp_output)

        colors = trainer.dataset.class_colormap[cloud_semantic_labels] / 255.0
        pcd_gt.colors = o3d.utility.Vector3dVector(colors)
        colors = trainer.dataset.class_colormap[cloud_semantic_output] / 255.0
        pcd_pred.colors = o3d.utility.Vector3dVector(colors)

        o3d.visualization.draw_geometries([pcd_pred, pcd_gt] + spheres)

        pcd_gt.points = o3d.utility.Vector3dVector(cloud_voxels)
        pcd_gt.translate((20 / VOXEL_SIZE, 0, 0))
        pcd_pred.points = o3d.utility.Vector3dVector(cloud_voxels)

        colors = np.array([id2color[i] for i in cloud_instance_labels], dtype=np.float64)
        pcd_gt.colors = o3d.utility.Vector3dVector(colors)
        colors = np.array([id2color[i] for i in cloud_instance_output_remap], dtype=np.float64)
        pcd_pred.colors = o3d.utility.Vector3dVector(colors)

        o3d.visualization.draw_geometries([pcd_pred, pcd_gt] + spheres)


Parámetros totales: 53,952,472
Parámetros entrenables: 53,952,472
Resnet generates features at the following scales:
	* (0.2, 0.2, 0.2) meters -> 32 feats.
	* (0.4, 0.4, 0.4) meters -> 64 feats.
	* (0.8, 0.8, 0.8) meters -> 128 feats.
	* (1.6, 1.6, 1.6) meters -> 256 feats.
	* (1.6, 1.6, 3.2) meters -> 256 feats.

Minimum scene size: (4.8, 4.8, 9.6) meters
Version name: tree_projector_VS-0.2_DA-48_E-3_V2

=== Starting epoch 1 ===
Training instance correlation with labels instead of predictions by now...

[Train]:   0%|          | 0/5520 [00:01<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (825x384 and 256x128)